<a href="https://colab.research.google.com/github/eceirem/Hukuki-Asistan/blob/hibrit_RRF_Cross-Encoder_BM25/hibrit_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uygulama ve Arama Motoru (Demonstrasyon)

**Amacı: **Bir ölçüm yapmaktan ziyade, sistemin canlı bir arama motoru gibi nasıl çalıştığını göstermek. İlk dökümanın özetini alır ve ona en yakın 5 emsali bulup ekrana yazdırır.

**Kullanılan Yöntemler: **Segment ayıklama (extract_sections), RRF Fusion ve Cross-Encoder Reranking.

In [ ]:
# ===========================================================
# HUKUKİ ASİSTAN - GELİŞMİŞ HİBRİT EMSAL ARAMA MOTORU (v2)
# Mimari: BM25 + Bi-Encoder -> RRF Fusion -> Cross-Encoder Rerank
# ===========================================================

import os, glob, json, re, random
import numpy as np
import torch
from tqdm import tqdm
from typing import Dict, Any, List, Tuple, Optional


!pip install -q rank-bm25 sentence-transformers tqdm

from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. CONFIG & PATHS ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements" # Google Drive yoluna göre ayarlı [cite: 43]
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # [cite: 123, 208]
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2" # Yeniden sıralama için usta dokunuşu

# --- 2. VERİ YÜKLEME VE TEMİZLEME ---
def clean_text(s: Any) -> str:
    s = "" if s is None else str(s)
    return re.sub(r"\s+", " ", s).strip()

def load_legal_corpus(data_dir: str):
    paths = sorted(glob.glob(os.path.join(data_dir, "*.json")))
    raw_objs, doc_ids = [], []
    for p in tqdm(paths, desc="JSON Kararlar Yükleniyor"):
        with open(p, "r", encoding="utf-8") as f:
            raw_objs.append(json.load(f))
        doc_ids.append(os.path.basename(p))
    return raw_objs, doc_ids

# Karar bölümlerini makaleye uygun şekilde ayıklıyoruz [cite: 38, 99]
def extract_sections(obj: Dict[str, Any]):
    rrl = obj.get("rrl_segments") or {}
    sf  = obj.get("structural_features") or {}
    md  = obj.get("meta_data") or {}

    return {
        "summary": clean_text(obj.get("summary_for_model")), # Sorgu olarak kullanılacak [cite: 126]
        "facts": clean_text(rrl.get("facts_text")),
        "reasoning": clean_text(rrl.get("reasoning_text")), # En ayırt edici bölüm [cite: 15, 190]
        "laws": " ".join([clean_text(x) for x in sf.get("mentioned_laws", [])]),
        "full_index": f"{rrl.get('facts_text', '')} {rrl.get('reasoning_text', '')} {sf.get('mentioned_laws', '')}".strip()
    }

# --- 3. ARAMA VE SIRALAMA ALGORİTMALARI ---

# RRF: İki farklı modelin sıralamasını adil şekilde birleştirir
def rrf_fusion(ranks_list: List[np.ndarray], k: int = 60):
    rrf_scores = np.zeros(ranks_list[0].shape)
    for ranks in ranks_list:
        rrf_scores += 1.0 / (k + ranks + 1)
    return rrf_scores

# Cross-Encoder: İlk 50 adayı derinlemesine analiz eder
def apply_reranking(ce_model, query, candidates, top_k=10):
    pairs = [[query, cand] for cand in candidates]
    scores = ce_model.predict(pairs)
    return np.argsort(-scores)[:top_k]

# --- 4. ANA ÇALIŞTIRICI (MAIN PIPELINE) ---

def main():
    # Ensure Google Drive is mounted
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
    except ImportError:
        print("Google Colab environment not detected. Skipping drive mount.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")

    # 1. Veriyi Oku
    raw_objs, doc_ids = load_legal_corpus(DATA_DIR)

    if not raw_objs:
        print(f"UYARI: '{DATA_DIR}' dizininden hiç hukuki karar dosyası yüklenemedi. Lütfen yolun doğru olduğundan ve JSON dosyalarının mevcut olduğundan emin olun.")
        return # Exit if no data is loaded

    processed_data = [extract_sections(o) for o in raw_objs]

    # Filter out entries where 'reasoning' is empty, as this is used for corpus_texts
    filtered_data = [d for i, d in enumerate(processed_data) if d["reasoning"] and doc_ids[i] is not None]
    filtered_doc_ids = [doc_ids[i] for i, d in enumerate(processed_data) if d["reasoning"] and doc_ids[i] is not None]

    if not filtered_data:
        print("UYARI: 'Gerekçe' (Reasoning) bölümü içeren geçerli metin bulunamadı. BM25 ve Bi-Encoder indekslemesi yapılamıyor.")
        return # Exit if no valid data for corpus

    # Boş olmayan sorguları ve dökümanları ayır [cite: 101]
    # 'summary' alanı boş olmayanları alıyoruz
    queries = [d["summary"] for d in filtered_data if d["summary"]]

    if not queries:
        print("UYARI: Sorgu için kullanılabilecek geçerli 'summary' alanı bulunamadı. Test sorgusu oluşturulamıyor.")
        return # Exit if no queries can be formed

    corpus_texts = [d["reasoning"] for d in filtered_data]

    # 2. BM25 Hazırla (Kelimeden yakalar) [cite: 118, 119]
    tokenized_corpus = [re.findall(r"\w+", text.lower()) for text in corpus_texts]
    bm25 = BM25Okapi(tokenized_corpus)

    # 3. Bi-Encoder Hazırla (Anlamdan yakalar) [cite: 122, 210]
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)
    doc_embeddings = bi_model.encode(corpus_texts, convert_to_numpy=True, show_progress_bar=True)
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)

    # 4. Örnek Bir Sorgu Üzerinde Test Et
    test_query = queries[0]
    print(f"\nSorgu (Summary for Model): {test_query[:150]}...\n")

    # A) BM25 Skorları ve Sıralaması
    bm25_scores = bm25.get_scores(re.findall(r"\w+", test_query.lower()))
    bm25_ranks = np.argsort(np.argsort(-bm25_scores))

    # B) Dense (Bi-Encoder) Skorları ve Sıralaması
    query_emb = bi_model.encode(test_query)
    dense_scores = query_emb @ doc_embeddings.T
    dense_ranks = np.argsort(np.argsort(-dense_scores))

    # C) RRF Hibrit Birleştirme (Yenilik!) [cite: 141, 194]
    combined_scores = rrf_fusion([bm25_ranks, dense_ranks])
    top_50_indices = np.argsort(-combined_scores)[:50]

    # D) Cross-Encoder Reranking (En Üst Düzey Hassasiyet)
    candidate_texts = [corpus_texts[idx] for idx in top_50_indices]
    reranked_sub_indices = apply_reranking(ce_model, test_query, candidate_texts, top_k=5)
    final_indices = top_50_indices[reranked_sub_indices]

    # 5. Sonuçları Yazdır
    print("--- EN YAKIN EMSAL KARARLAR ---")
    for i, idx in enumerate(final_indices):
        print(f"{i+1}. Dosya: {filtered_doc_ids[idx]} | Skor (RRF): {combined_scores[idx]:.4f}")
        print(f"   Gerekçe Özeti: {corpus_texts[idx][:200]}...\n")

if __name__ == "__main__":
    main()

Mounted at /content/drive


JSON Kararlar Yükleniyor: 100%|██████████| 1000/1000 [00:35<00:00, 27.78it/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]


Sorgu (Summary for Model): Davacı vekili, davalı şirketin sosyal medya hizmetleri karşılığında fatura düzenlediğini ancak ücretin ödenmediğini iddia etmiştir. Mahkeme, davacı ve...

--- EN YAKIN EMSAL KARARLAR ---
1. Dosya: vision_llm_processed_2017_806.json | Skor (RRF): 0.0236
   Gerekçe Özeti: Davacı vekili, davacı şirketin sosyal medyada yaptığı paylaşımda eleştiri ve serserze bulunduğunu, müvekkilinin yapmış olduğu paylaşımın tamamen kişilik haklarına saldırdığını veya hakaret içermediğin...

2. Dosya: vision_llm_processed_2019_3432.json | Skor (RRF): 0.0155
   Gerekçe Özeti: İlk derece mahkemesi, davacı vekili tarafından istinaf edilmiş olan davanın, vekil ile davalı arasında sözleşme bağlandığı ve vekilin hizmetlerini yerine getirdiği halde, davalı firmanın vekil şirkete...

3. Dosya: vision_llm_processed_2021_542.json | Skor (RRF): 0.0221
   Gerekçe Özeti: Davacı vekili, davalı şirketin müvekkiline satılan malların faturalarını kabul etmediğini, ancak bu faturaların müvekkilce

# Performans Ölçüm Paneli (Hızlı Benchmark)

**Amacı: **Kod 1 ile neredeyse aynıdır ancak veri yükleme ve işleme kısımları daha sadedir. Kod 1'in daha doğrudan ve hızlı çalışan bir versiyonudur.

**Kullanılan Yöntemler: **BM25, Bi-Encoder ve Cross-Encoder.

In [ ]:
import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm # Döngüleri izlemek için usta dokunuşu
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. AYARLAR VE YOLLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# --- 2. EVALUATION FONKSİYONU ---
def calculate_recall_at_k(ranks, k_values=[1, 5, 10]):
    results = {}
    for k in k_values:
        hits = sum(1 for r in ranks if r < k)
        results[f"R@{k}"] = round(hits / len(ranks), 4)
    return results

# --- 3. ANA BENCHMARK FONKSİYONU ---
def run_benchmark():
    # A) Dosya Yükleme
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    corpus_data = []

    for p in tqdm(paths, desc="1. Dosyalar Diskten Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            rrl = obj.get("rrl_segments", {})
            corpus_data.append({
                "id": os.path.basename(p),
                "query": obj.get("summary_for_model", ""),
                "reasoning": rrl.get("reasoning_text", ""),
            })

    # B) Filtreleme
    valid_data = [d for d in corpus_data if d["query"] and d["reasoning"]]
    queries = [d["query"] for d in valid_data]
    docs = [d["reasoning"] for d in valid_data]
    total = len(queries)

    print(f"\n✅ Veri Hazır: {total} geçerli döküman/sorgu bulundu.\n")

    # --- MODEL 1: BM25 (Sparse) ---
    print("🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...")
    tokenized_corpus = [re.findall(r"\w+", d.lower()) for d in docs]
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_ranks = []
    for i, q in tqdm(enumerate(queries), total=total, desc="   BM25 Sıralanıyor"):
        scores = bm25.get_scores(re.findall(r"\w+", q.lower()))
        rank = np.argsort(-scores).tolist().index(i)
        bm25_ranks.append(rank)

    results_bm25 = calculate_recall_at_k(bm25_ranks)
    results_bm25["Model"] = "BM25 (Reasoning)"

    # --- MODEL 2: BI-ENCODER (Dense) ---
    print("\n🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...")
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)

    # Encoding aşamasını izle
    print("   Vektörler oluşturuluyor (Bu aşama GPU hızına bağlıdır)...")
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)

    bi_ranks = []
    for i in tqdm(range(total), desc="   Vektörler Karşılaştırılıyor"):
        scores = query_embs[i] @ doc_embs.T
        rank = np.argsort(-scores).tolist().index(i)
        bi_ranks.append(rank)

    results_bi = calculate_recall_at_k(bi_ranks)
    results_bi["Model"] = "Dense (MPNet)"

    # --- MODEL 3: HİBRİT + CROSS-ENCODER (Usta İşi) ---
    print("\n🚀 Adım 3: Hibrit + Cross-Encoder Re-rank Başlıyor...")
    print("   ⚠️ UYARI: Bu aşama en ağır olanıdır, derinlemesine analiz yapar.")
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
    final_ranks = []
    K = 60

    # Ana döngüyü izle
    for i in tqdm(range(total), desc="   Derin Analiz (Re-ranking)"):
        # 1. BM25 & Dense skorlarını al
        b_scores = bm25.get_scores(re.findall(r"\w+", queries[i].lower()))
        b_rank_indices = np.argsort(-b_scores)
        d_scores = query_embs[i] @ doc_embs.T
        d_rank_indices = np.argsort(-d_scores)

        # 2. RRF Fusion
        rrf_scores = np.zeros(total)
        for rank, idx in enumerate(b_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)
        for rank, idx in enumerate(d_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)

        # 3. İlk 20 adayı Cross-Encoder'a gönder
        top_20_candidates = np.argsort(-rrf_scores)[:20]
        pairs = [[queries[i], docs[idx]] for idx in top_20_candidates]
        ce_scores = ce_model.predict(pairs, show_progress_bar=False) # Her adımda iç bar gerekmez

        # 4. Final Sıralama
        best_within_top20 = np.argsort(-ce_scores)
        final_top_indices = top_20_candidates[best_within_top20]

        if i in final_top_indices:
            rank = final_top_indices.tolist().index(i)
        else:
            rank = 999
        final_ranks.append(rank)

    results_hybrid = calculate_recall_at_k(final_ranks)
    results_hybrid["Model"] = "Bizim Hibrit (RRF + CE)"

    # --- SONUÇLARI TABLOLAŞTIR ---
    df = pd.DataFrame([results_bm25, results_bi, results_hybrid])
    print("\n" + "="*50)
    print("📊 TEST SONUÇLARI TAMAMLANDI")
    print("="*50)
    return df[["Model", "R@1", "R@5", "R@10"]]

# --- ÇALIŞTIR ---
comparison_table = run_benchmark()
print(comparison_table)

1. Dosyalar Diskten Okunuyor:   0%|          | 0/1000 [00:00<?, ?it/s]


✅ Veri Hazır: 970 geçerli döküman/sorgu bulundu.

🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...


   BM25 Sıralanıyor:   0%|          | 0/970 [00:00<?, ?it/s]


🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...
   Vektörler oluşturuluyor (Bu aşama GPU hızına bağlıdır)...


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

   Vektörler Karşılaştırılıyor:   0%|          | 0/970 [00:00<?, ?it/s]


🚀 Adım 3: Hibrit + Cross-Encoder Re-rank Başlıyor...
   ⚠️ UYARI: Bu aşama en ağır olanıdır, derinlemesine analiz yapar.


   Derin Analiz (Re-ranking):   0%|          | 0/970 [00:00<?, ?it/s]


📊 TEST SONUÇLARI TAMAMLANDI
                     Model     R@1     R@5    R@10
0         BM25 (Reasoning)  0.7299  0.8278  0.8598
1            Dense (MPNet)  0.2856  0.4165  0.4866
2  Bizim Hibrit (RRF + CE)  0.2990  0.5082  0.6381


# Geliştirilmiş Benchmark Kodu

**Amacı:** Sistemdeki tüm modelleri (BM25, Dense ve Hibrit) veri setinin tamamında test edip "Hangi model daha başarılı?" sorusuna bilimsel (Recall@K) yanıt vermek.

**Kullanılan Yöntemler:** BM25 (Kelime bazlı), MPNet (Anlamsal/Vektörel), RRF (Sıralama Birleştirme) ve Cross-Encoder (Yeniden Sıralama).

**Öne Çıkan Özelliği:** Hata payını minimize eden temizlik fonksiyonları (clean_text) ve dökümanların listede bulunamaması durumuna karşı korumalı bir yapı içerir.

In [ ]:
import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm # Döngüleri izlemek için usta dokunuşu
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder
from typing import Any
from google.colab import drive # Import drive for mounting

# --- 1. AYARLAR VE YOLLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BI_ENCODER_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CROSS_ENCODER_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# --- Yardımcı Fonksiyon: Metin Temizleme ---
def clean_text(s: Any) -> str:
    s = "" if s is None else str(s)
    return re.sub(r"\s+", " ", s).strip()

# --- 2. EVALUATION FONKSİYONU ---
def calculate_recall_at_k(ranks, k_values=[1, 5, 10]):
    results = {}
    for k in k_values:
        hits = sum(1 for r in ranks if r < k)
        results[f"R@{k}"] = round(hits / len(ranks), 4)
    return results

# --- 3. ANA BENCHMARK FONKSİYONU ---
def run_benchmark():
    # Ensure Google Drive is mounted
    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"]) # Exit if mount fails

    # A) Dosya Yükleme
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    corpus_data = []

    if not paths:
        print(f"UYARI: '{DATA_DIR}' dizininde hiç hukuki karar dosyası bulunamadı.")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"])

    for p in tqdm(paths, desc="1. Dosyalar Diskten Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            rrl = obj.get("rrl_segments", {})
            corpus_data.append({
                "id": os.path.basename(p),
                "query": clean_text(obj.get("summary_for_model", "")),
                "reasoning": clean_text(rrl.get("reasoning_text", "")),
            })

    # B) Filtreleme
    valid_data = [d for d in corpus_data if d["query"] and d["reasoning"]]
    queries = [d["query"] for d in valid_data]
    docs = [d["reasoning"] for d in valid_data]
    total = len(queries)

    print(f"\n✅ Veri Hazır: {total} geçerli döküman/sorgu bulundu.\n")

    if total == 0:
        print("UYARI: Hiç geçerli sorgu veya doküman bulunamadı. Benchmark çalıştırılamıyor.")
        return pd.DataFrame(columns=["Model", "R@1", "R@5", "R@10"]) # Boş DataFrame döndür

    # --- MODEL 1: BM25 (Sparse) ---
    print("🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...")
    tokenized_corpus = [re.findall(r"\w+", d.lower()) for d in docs]
    bm25 = BM25Okapi(tokenized_corpus)
    bm25_ranks = []
    for i, q in tqdm(enumerate(queries), total=total, desc="   BM25 Sıralanıyor"):
        tokenized_query = re.findall(r"\w+", q.lower())
        if not tokenized_query: # Boş sorguları ele al
            bm25_ranks.append(999) # Çok düşük bir sıralama ver
            continue
        scores = bm25.get_scores(tokenized_query)
        rank_list = np.argsort(-scores).tolist()
        # Eğer mevcut döküman (i) sıralama listesinde değilse veya BM25 sonucu anlamsızsa
        # Güvenli erişim için kontrol ekle: i'nin rank_list içinde olduğundan emin ol
        if i < len(rank_list) and i in rank_list:
            rank = rank_list.index(i)
        else:
            rank = 999
        bm25_ranks.append(rank)

    results_bm25 = calculate_recall_at_k(bm25_ranks)
    results_bm25["Model"] = "BM25 (Reasoning)"

    # --- MODEL 2: BI-ENCODER (Dense) ---
    print("\n🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...")
    bi_model = SentenceTransformer(BI_ENCODER_MODEL, device=DEVICE)

    # Encoding aşamasını izle
    print("   Vektörler oluşturuluyor (Bu aşama GPU hızına bağlıdır)...")
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)

    bi_ranks = []
    for i in tqdm(range(total), desc="   Vektörler Karşılaştırılıyor"):
        scores = query_embs[i] @ doc_embs.T
        rank = np.argsort(-scores).tolist().index(i)
        bi_ranks.append(rank)

    results_bi = calculate_recall_at_k(bi_ranks)
    results_bi["Model"] = "Dense (MPNet)"

    # --- MODEL 3: HİBRİT + CROSS-ENCODER (Usta İşi) ---
    print("\n🚀 Adım 3: Hibrit + Cross-Encoder Re-rank Başlıyor...")
    print("   ⚠️ UYARI: Bu aşama en ağır olanıdır, derinlemesine analiz yapar.")
    ce_model = CrossEncoder(CROSS_ENCODER_MODEL, device=DEVICE)
    final_ranks = []
    K = 60

    # Ana döngüyü izle
    for i in tqdm(range(total), desc="   Derin Analiz (Re-ranking)"):
        # 1. BM25 & Dense skorlarını al
        b_scores = bm25.get_scores(re.findall(r"\w+", queries[i].lower()))
        b_rank_indices = np.argsort(-b_scores)
        d_scores = query_embs[i] @ doc_embs.T
        d_rank_indices = np.argsort(-d_scores)

        # 2. RRF Fusion
        rrf_scores = np.zeros(total)
        # Boş BM25 sorgularını ele al, aksi takdirde rrf_scores'a katkı sağlamaz
        if re.findall(r"\w+", queries[i].lower()): # Yalnızca token'lar varsa katkı sağla
            for rank, idx in enumerate(b_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)
        for rank, idx in enumerate(d_rank_indices): rrf_scores[idx] += 1 / (K + rank + 1)

        # 3. İlk 20 adayı Cross-Encoder'a gönder
        top_20_candidates = np.argsort(-rrf_scores)[:20]
        pairs = [[queries[i], docs[idx]] for idx in top_20_candidates]
        ce_scores = ce_model.predict(pairs, show_progress_bar=False) # Her adımda iç bar gerekmez

        # 4. Final Sıralama
        best_within_top20 = np.argsort(-ce_scores)
        final_top_indices = top_20_candidates[best_within_top20]

        if i in final_top_indices:
            rank = final_top_indices.tolist().index(i)
        else:
            rank = 999
        final_ranks.append(rank)

    results_hybrid = calculate_recall_at_k(final_ranks)
    results_hybrid["Model"] = "Bizim Hibrit (RRF + CE)"

    # --- SONUÇLARI TABLOLAŞTIR ---
    df = pd.DataFrame([results_bm25, results_bi, results_hybrid])
    print("\n" + "="*50)
    print("📊 TEST SONUÇLARI TAMAMLANDI")
    print("="*50)
    return df[["Model", "R@1", "R@5", "R@10"]]

# --- ÇALIŞTIR ---
comparison_table = run_benchmark()
print(comparison_table)

Mounted at /content/drive


1. Dosyalar Diskten Okunuyor:   0%|          | 0/1000 [00:00<?, ?it/s]


✅ Veri Hazır: 970 geçerli döküman/sorgu bulundu.

🚀 Adım 1: BM25 (Kelime Bazlı) Analizi Başlıyor...


   BM25 Sıralanıyor:   0%|          | 0/970 [00:00<?, ?it/s]


🚀 Adım 2: Bi-Encoder (Anlamsal) Analizi Başlıyor...


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

   Derin Analiz (Re-ranking):   0%|          | 0/970 [00:00<?, ?it/s]


📊 TEST SONUÇLARI TAMAMLANDI
                     Model     R@1     R@5    R@10
0         BM25 (Reasoning)  0.7299  0.8278  0.8598
1            Dense (MPNet)  0.2856  0.4165  0.4866
2  Bizim Hibrit (RRF + CE)  0.2990  0.5082  0.6381


# ULTIMATE BENCHMARK (PARM + CROSS-ENCODER)

**Amacı:** Uzun hukuk metinlerindeki "anlam seyrelmesi" sorununu aşmak için dökümanları parçalara bölerek (chunking) en hassas ve derinlemesine emsal tespitini gerçekleştirir.

**Kullanılan Yöntemler:** Paragraph Chunking, PARM (MaxSim Pooling) ve Cross-Encoder Reranking.

In [7]:
# ============================================================
# HUKUKİ ASİSTAN - ULTIMATE BENCHMARK (PARM + CROSS-ENCODER)
# Mimari: BM25, Bi-Encoder, PARM (Chunking) ve Cross-Encoder
# ============================================================

import os, glob, json, re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import torch
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

# --- 1. AYARLAR ---
DATA_DIR = "/content/drive/MyDrive/Legal NLP/Dataset/Structured_Judgements"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BI_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
CE_MODEL = "cross-encoder/ms-marco-MiniLM-L-6-v2"

# 2. CHUNKING FONKSİYONU (PARM'ın Kalbi)
def get_chunks(text, chunk_size=250):
    """Metni paragraf benzeri küçük parçalara böler."""
    words = text.split()
    if not words: return [""]
    return [" ".join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]

def run_ultimate_benchmark():
    # A) VERİ YÜKLEME
    paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.json")))
    all_data = []

    for p in tqdm(paths, desc="1. Dosyalar Okunuyor"):
        with open(p, "r", encoding="utf-8") as f:
            obj = json.load(f)
            reasoning = obj.get("rrl_segments", {}).get("reasoning_text", "")
            summary = obj.get("summary_for_model", "")
            if summary and reasoning:
                all_data.append({
                    "id": os.path.basename(p),
                    "query": summary,
                    "doc": reasoning,
                    "chunks": get_chunks(reasoning) # PARM için dökümanı böldük
                })

    total = len(all_data)
    queries = [d["query"] for d in all_data]
    docs = [d["doc"] for d in all_data]

    # MODELLERİ YÜKLE
    bi_model = SentenceTransformer(BI_MODEL, device=DEVICE)
    ce_model = CrossEncoder(CE_MODEL, device=DEVICE)

    # B) VEKTÖRLEŞTİRME
    print("\n🚀 Adım 2: Vektörler Oluşturuluyor...")
    query_embs = bi_model.encode(queries, convert_to_numpy=True, show_progress_bar=True)
    doc_embs = bi_model.encode(docs, convert_to_numpy=True, show_progress_bar=True)

    # --- MODEL: PARM (İNOVATİF) ---
    print("\n🚀 Adım 3: PARM (Paragraph Aggregation) Ölçülüyor...")
    parm_ranks = []
    for i in tqdm(range(total), desc="   PARM Analizi"):
        q_emb = query_embs[i]
        doc_max_scores = []
        for d_entry in all_data:
            # Dökümanın her bir chunk'ını sorguyla kıyasla
            chunk_embs = bi_model.encode(d_entry["chunks"], convert_to_numpy=True, show_progress_bar=False)
            chunk_scores = q_emb @ chunk_embs.T
            doc_max_scores.append(np.max(chunk_scores)) # MAX POOLING: En iyi paragrafın puanını al
        parm_ranks.append(np.argsort(-np.array(doc_max_scores)).tolist().index(i))

    # --- MODEL: HİBRİT + CROSS-ENCODER ---
    print("\n🚀 Adım 4: Hibrit + Cross-Encoder Ölçülüyor...")
    final_ranks = []
    for i in tqdm(range(total), desc="   Re-ranking Analizi"):
        initial_scores = query_embs[i] @ doc_embs.T
        top_20_indices = np.argsort(-initial_scores)[:20]

        candidates = [docs[idx] for idx in top_20_indices]
        pairs = [[queries[i], cand[:1000]] for cand in candidates] # İlk 1000 karakter (Hız için)
        ce_scores = ce_model.predict(pairs, show_progress_bar=False)

        reranked_indices = top_20_indices[np.argsort(-ce_scores)]
        final_ranks.append(reranked_indices.tolist().index(i) if i in reranked_indices else 999)

    # SONUÇLARI TABLOLAŞTIR
    def get_metrics(ranks, name):
        return {"Model": name,
                "R@1": round(sum(1 for r in ranks if r < 1)/len(ranks), 4),
                "R@5": round(sum(1 for r in ranks if r < 5)/len(ranks), 4),
                "R@10": round(sum(1 for r in ranks if r < 10)/len(ranks), 4)}

    df = pd.DataFrame([
        get_metrics(parm_ranks, "PARM (Paragraph Aggregation)"),
        get_metrics(final_ranks, "Hibrit + Cross-Encoder (CE)")
    ])
    return df

# ÇALIŞTIR
results = run_ultimate_benchmark()
print(results)

1. Dosyalar Okunuyor:   0%|          | 0/995 [00:00<?, ?it/s]


🚀 Adım 2: Vektörler Oluşturuluyor...


Batches:   0%|          | 0/31 [00:00<?, ?it/s]

Batches:   0%|          | 0/31 [00:00<?, ?it/s]


🚀 Adım 3: PARM (Paragraph Aggregation) Ölçülüyor...


   PARM Analizi:   0%|          | 0/982 [00:00<?, ?it/s]

KeyboardInterrupt: 

Burada ne yapmaya çalıştığımı hatırlamıyorum

In [ ]:
# Data işleme, LLM ve QLoRA için
!pip install transformers accelerate bitsandbytes peft trl datasets
# Retrieval (XLM-R Ailesi) için
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.4/517.4 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 86.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import json
from datasets import Dataset
import os          # Klasör ve dosya yollarını yönetmek için
import glob        # Belirli uzantıdaki dosyaları (örneğin *.json) bulmak için
from google.colab import drive

In [ ]:
import os
import glob
import pandas as pd
import json
from google.colab import drive # Drive mount işlemi için

# -----------------------------------------------------------
# KLASÖR YOLLARI
# -----------------------------------------------------------

drive.mount('/content/drive', force_remount=True)

# 1. Adım: Tam yolu doğru şekilde birleştirin
DRIVE_BASE_DIR = "/content/drive/MyDrive"

# Göreli yollarınızı birleştirin
RELATIVE_PATH = "Legal NLP/Dataset/Structured_Judgements"

# TAM YOL (Absolute Path)
# Python'a, dosyaların nerede olduğunu tam olarak söylemeliyiz.
FULL_DATA_PATH = os.path.join(DRIVE_BASE_DIR, RELATIVE_PATH)


# -----------------------------------------------------------
# 1. Veri Setini Yükleme ve Birleştirme (YENİDEN)
# -----------------------------------------------------------

# Klasör içindeki tüm *.json dosyalarının yolunu listeleme
# TAM YOL'u kullanarak arama yapıyoruz.
all_files = glob.glob(os.path.join(FULL_DATA_PATH, '*.json'))

# Kontrol: Kaç dosya bulduğunu yazdırın
print(f"**BULUNAN DOSYA SAYISI:** {len(all_files)}")

data_list = []
# Gerçek proje: for file_name in klasör_deki_dosyalar:
for file_path in all_files:
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            data_list.append(data)
    except Exception as e:
        # Hata yönetimi: Bir dosya bozuksa tüm süreci durdurmasın.
        print(f"Hata! {file_path} dosyası yüklenemedi. Hata: {e}")
        continue # Sonraki dosyaya geç

# Yüklenen veriyi DataFrame'e dönüştürme
df = pd.DataFrame(data_list)

# -----------------------------------------------------------
# 2. Karar Metinlerini Birleştirme (Full Text Oluşturma)
# -----------------------------------------------------------

# Gerekli Sütunları Çekme (Emsal Karar için en önemli bilgileri birleştiriyoruz)
# Bu kısım, tüm dosyalar yüklendikten sonra toplu olarak çalışır.
df['full_text'] = df['meta_data'].apply(lambda x: x.get('case_subject', '')) + \
                   " | Esas No: " + df['meta_data'].apply(lambda x: x.get('esas_no', '')) + \
                   " | Konusu: " + df['rrl_segments'].apply(lambda x: x.get('facts_text', '')) + \
                   " | Gerekçe: " + df['rrl_segments'].apply(lambda x: x.get('reasoning_text', ''))

# -----------------------------------------------------------
# 3. Hugging Face Dataset Nesnesine Dönüştürme
# -----------------------------------------------------------
corpus_df = df[['meta_data', 'full_text']].copy()
corpus_dataset = Dataset.from_pandas(corpus_df)

print("\nVeri Hazırlığı Tamamlandı.")
print(f"Oluşturulan veri setinde {len(corpus_dataset)} adet örnek bulunmaktadır.")

Mounted at /content/drive
**BULUNAN DOSYA SAYISI:** 1000

Veri Hazırlığı Tamamlandı.
Oluşturulan veri setinde 1000 adet örnek bulunmaktadır.


In [ ]:
# SBERT: XLM-R benzeri anlamsal vektörleme için
# FAISS: Vektör veritabanı (çok hızlı arama) için
!pip install sentence-transformers faiss-cpu

In [ ]:
# ==============================================================================
# AŞAMA 0: KURULUM VE KÜTÜPHANELER
# Bu kısım, tüm gerekli kütüphaneleri yükler ve Google Drive bağlantısını kurar.
# ==============================================================================
import os
import glob
import json
import torch
import pandas as pd
import numpy as np
from google.colab import drive
from datasets import Dataset

# Retrieval (Erişim) için gerekli kütüphaneler (XLM-R ailesi)
from sentence_transformers import SentenceTransformer
import faiss

# Generation (Üretim) ve QLoRA için gerekli kütüphaneler (Qwen)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer

# --- KÜTÜPHANE KURULUMU (Colab'de bir kez çalıştırılmalı) ---
# Yorum satırlarını kaldırıp çalıştırın.
# !pip install -q transformers accelerate bitsandbytes peft trl datasets sentence-transformers faiss-cpu

# --- GOOGLE DRIVE BAĞLANTISI ---
# Veri setinizin olduğu Drive klasörüne erişmek için gereklidir.
drive.mount('/content/drive', force_remount=True)

# KLASÖR YOLLARI
# Lütfen bu yolları kendi Drive klasör yapınıza göre ayarlayın.
DRIVE_BASE_DIR = "/content/drive/MyDrive"
RELATIVE_PATH = "Legal NLP/Dataset/Structured_Judgements"
FULL_DATA_PATH = os.path.join(DRIVE_BASE_DIR, RELATIVE_PATH)
FAISS_INDEX_PATH = "hukuk_faiss_index.bin"
ADAPTER_SAVE_PATH = "./hukuk_qwen_adaptoru"

# ==============================================================================
# AŞAMA 1: VERİ HAZIRLIĞI VE FORMATLAMA
# 1000 JSON dosyasını okur ve hem arama hem de LLM eğitimi için tek bir formata dönüştürür.
# ==============================================================================

def prepare_data():
    """Tüm JSON dosyalarını okur ve Pandas DataFrame'e dönüştürür."""
    print("--- 1. Veri Seti Yükleme ---")

    # 1. Dosya Yollarını Bulma
    all_files = glob.glob(os.path.join(FULL_DATA_PATH, '*.json'))
    print(f"**BULUNAN DOSYA SAYISI:** {len(all_files)}")

    data_list = []
    # 2. Dosyaları Tek Tek Okuma
    for file_path in all_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data_list.append(json.load(f))
        except Exception as e:
            # Hata yönetimi: Bozuk dosyaları atla
            print(f"Hata! {file_path} dosyası yüklenemedi. Hata: {e}")
            continue

    if not data_list:
        raise ValueError("HATA: Hiçbir dosya başarıyla yüklenemedi. Lütfen 'FULL_DATA_PATH' yolunu kontrol edin.")

    # 3. DataFrame'e Dönüştürme
    df = pd.DataFrame(data_list)

    # 4. Arama ve LLM için Birleştirilmiş Metni Oluşturma (full_text)
    # Bu metin, kararın en önemli kısımlarını içerir ve Retrieval modeli bunu vektörleyecektir.
    df['full_text'] = df['meta_data'].apply(lambda x: x.get('case_subject', '') if isinstance(x, dict) else '') + \
                       " | Esas No: " + df['meta_data'].apply(lambda x: x.get('esas_no', '') if isinstance(x, dict) else '') + \
                       " | Konusu: " + df['rrl_segments'].apply(lambda x: x.get('facts_text', '') if isinstance(x, dict) else '') + \
                       " | Gerekçe: " + df['rrl_segments'].apply(lambda x: x.get('reasoning_text', '') if isinstance(x, dict) else '')

    # 5. Hugging Face Dataset Nesnesine Dönüştürme
    # Transformer kütüphaneleri bu formatı bekler.
    corpus_df = df[['meta_data', 'full_text', 'rrl_segments']].copy() # rrl_segments LLM çıktısı için saklanır
    corpus_dataset = Dataset.from_pandas(corpus_df)

    print(f"Veri Hazırlığı Tamamlandı. Dataset boyutu: {len(corpus_dataset)} örnek.")
    return corpus_dataset

# ==============================================================================
# AŞAMA 2: ERİŞİM (RETRIEVAL) MODELİ KURULUMU (XLM-R/SBERT + FAISS)
# Kullanıcının sorgusu ile 1000 karar arasındaki anlamsal benzerliği tespit eder.
# ==============================================================================

def setup_retrieval_system(corpus_dataset):
    """XLM-R ailesi (SBERT) modelini kullanarak FAISS indeksini oluşturur."""
    print("\n--- 2. Erişim Sistemi Kurulumu (XLM-R/FAISS) ---")

    # 1. Retrieval Modelini Yükleme (XLM-R'ın SBERT uygulaması)
    retrieval_model_name = 'paraphrase-multilingual-mpnet-base-v2'
    retrieval_model = SentenceTransformer(retrieval_model_name)
    print(f"Retrieval Modeli yüklendi: {retrieval_model_name}")

    # 2. Hukuki Kararları Vektörlere Dönüştürme (Embedding)
    corpus_texts = corpus_dataset['full_text']
    corpus_embeddings = retrieval_model.encode(
        corpus_texts,
        convert_to_tensor=False,
        show_progress_bar=True
    )
    print(f"Vektör boyutu: {corpus_embeddings.shape}")

    # 3. FAISS Vektör Veritabanı Oluşturma
    embedding_dim = corpus_embeddings.shape[1]
    index = faiss.IndexFlatL2(embedding_dim)  # L2 (Öklid) uzaklığına göre indexleme
    index.add(corpus_embeddings.astype('float32'))
    faiss.write_index(index, FAISS_INDEX_PATH)
    print(f"FAISS İndeksi başarıyla oluşturuldu ve '{FAISS_INDEX_PATH}' adresine kaydedildi.")

    return retrieval_model, index

# --- Arama Fonksiyonu (RAG sisteminin ilk adımı) ---
def emsal_ara(query, retrieval_model, index, corpus_dataset, k=3):
    """Verilen sorgu için FAISS index'te en alakalı k kararı arar."""
    # 1. Sorguyu vektöre dönüştür
    query_embedding = retrieval_model.encode(query, convert_to_tensor=False)

    # 2. Arama yap (En yakın k kararın index numaralarını bul)
    D, I = index.search(query_embedding.astype('float32').reshape(1, -1), k)

    # 3. Bulunan index numaralarını Python int tipine dönüştür (Hata çözümü için kritik)
    retrieved_indices = [int(i) for i in I[0]]

    # 4. HuggingFace Dataset'ten orijinal veriyi çek
    retrieved_cases = [corpus_dataset[i] for i in retrieved_indices]

    return retrieved_cases

# ==============================================================================
# AŞAMA 3: ÜRETİM (GENERATION) MODELİ EĞİTİMİ (Qwen + QLoRA)
# Qwen modelini hukuki metinleri analiz etme ve özetleme yeteneği kazanması için özelleştirir.
# ==============================================================================

def create_instruction_format(row):
    """LLM eğitimi için veriyi Talimat-Yanıt (Instruction-Output) formatına dönüştürür."""
    # Talimat: Modelden beklentimiz, kararı özetlemesi.
    instruction = f"Esas No: {row['meta_data']['esas_no']} olan hukuki kararı analiz ederek, davaya konu olayı, mahkeme gerekçesini ve hükmünü detaylıca özetle."

    # Yanıt: Modelin öğrenmesini istediğimiz doğru çıktı (hukuki metinlerin birleşimi).
    output = (
        f"DAVANIN KONUSU: {row['meta_data']['case_subject']}\n\n"
        f"OLAY VE TALEPLER: {row['rrl_segments']['facts_text']}\n\n"
        f"MAHKEME GEREKÇESİ: {row['rrl_segments']['reasoning_text']}\n\n"
        f"HÜKÜM (VERDİCT): {row['rrl_segments']['verdict_text']}"
    )

    # Qwen gibi chat/instruction tabanlı modeller için standart eğitim formatı.
    return {"text": f"### HUKUKİ YÖNERGE:\n{instruction}\n\n### YANIT:\n{output}"}

def train_qwen_qlora(corpus_dataset):
    """Qwen modelini QLoRA tekniği ile hukuki veriler üzerinde eğitir."""
    print("\n--- 3. Üretim Modeli (Qwen + QLoRA) Eğitimi ---")

    # 1. Veri Setini LLM Eğitim Formatına Dönüştürme
    training_data = corpus_dataset.map(create_instruction_format)
    print("LLM Eğitim Veri Seti Hazırlandı.")

    # 2. QLoRA ve Qwen Ayarları
    model_id = "Qwen/Qwen1.5-7B-Chat" # Başlangıç modeli

    # QLoRA (4-bit) Ayarları: Bellek (VRAM) kullanımını büyük ölçüde azaltır.
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    # Modeli 4-bit nicelenmiş olarak yükleme
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
    model.config.use_cache = False
    model = prepare_model_for_kbit_training(model) # Kuantize modelin eğitim için hazırlanması

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    # LoRA Adaptör Ayarları: Sadece bu küçük matrisler eğitilecektir.
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules="all-linear",
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
    )

    # 3. Eğitimi Başlatma Argümanları
    training_args = TrainingArguments(
        output_dir="./qlora_hukuk_qwen_sonuc",
        num_train_epochs=3, # Veri setinin üzerinden 3 kez geçilmesi
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=2e-4, # Genellikle QLoRA için düşük öğrenme oranı kullanılır
        save_strategy="epoch",
        logging_steps=10
    )

    trainer = SFTTrainer(
        model=model,
        train_dataset=training_data,
        peft_config=lora_config,
        dataset_text_field="text",
        max_seq_length=2048, # Hukuki metinlerin uzunluğuna göre artırıldı
        tokenizer=tokenizer,
        args=training_args,
    )

    print("\n--- DİKKAT: QLoRA Hukuk Eğitimi Başlatılmak Üzere ---")
    print("Bu işlem, saatler sürebilir ve aktif bir GPU gerektirir.")

    # EĞİTİM KOMUTU (Başlatmak için yorum satırlarını kaldırın)
    # trainer.train()
    # trainer.model.save_pretrained(ADAPTER_SAVE_PATH)
    # tokenizer.save_pretrained(ADAPTER_SAVE_PATH)

    return model, tokenizer

# ==============================================================================
# AŞAMA 4: HİBRİT RAG SİSTEMİ (İNFERENCE/ÇIKTI)
# Erişim ve Üretim modellerini birleştirerek nihai yanıtı üretir.
# ==============================================================================

def setup_inference_model(adapter_path):
    """Eğitilmiş adaptörü yükler ve Qwen modeline birleştirir."""
    # 1. Base Model ve Tokenizer Yükleme
    base_model_id = "Qwen/Qwen1.5-7B-Chat"
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16)
    base_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(base_model_id)
    tokenizer.pad_token = tokenizer.eos_token

    # 2. Adaptörü Yükleme
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model = model.eval()
    print(f"Qwen Hukuk Asistanı adaptörü '{adapter_path}' adresinden başarıyla yüklendi.")
    return model, tokenizer

def hukuki_rag_sistemi(query, inference_model, tokenizer, retrieval_model, index, corpus_dataset):
    """Emsal arar ve bulunan bağlamı kullanarak yanıt üretir."""

    # A. Retrieval (Erişim): İlgili 3 Emsal Kararı Bulma
    print("\n[AŞAMA 1] İlgili Emsal Kararlar Aranıyor...")
    retrieved_cases = emsal_ara(query, retrieval_model, index, corpus_dataset, k=3)

    # Bağlam (Context) Metnini Oluşturma
    context = ""
    for i, case in enumerate(retrieved_cases):
        context += f"\n--- EMSAL KARAR {i+1} (Esas No: {case['meta_data']['esas_no']}) ---\n"
        context += case['full_text']

    # B. Generation (Üretim): Prompt (Yönerge) Hazırlama
    print("[AŞAMA 2] Qwen, Bağlamı Kullanarak Yanıt Üretmek İçin Hazırlanıyor...")

    # Modelden beklentimiz: Bulunan bağlama sadık kalarak, soruyu yanıtlaması.
    prompt = f"""
    Aşağıdaki Hukuki Bağlamı dikkatlice oku ve kullanıcının sorusuna yalnızca bu bağlamdaki bilgilere dayanarak, özetleyici ve açıklayıcı bir şekilde yanıt ver. Cevabının hukuki dayanağını (emsal kararları) belirt.

    ### KULLANICI SORUSU:
    {query}

    ### HUKUKİ BAĞLAM (EMSALLER):
    {context}

    ### YANIT:
    """

    # Modelden yanıt üretme (Inference)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

    with torch.no_grad():
        outputs = inference_model.generate(
            input_ids,
            max_new_tokens=512,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            eos_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
    return response

# ==============================================================================
# ANA ÇALIŞTIRMA BLOĞU
# ==============================================================================

if __name__ == "__main__":
    # --- 1. Veri Hazırlığı ---
    corpus_dataset = prepare_data()

    # --- 2. Erişim Sistemi Kurulumu ---
    retrieval_model, faiss_index = setup_retrieval_system(corpus_dataset)

    # --- 3. QLoRA Eğitimi ---
    # Eğer Qwen'i eğitecekseniz, yukarıdaki 'trainer.train()' yorumlarını kaldırın.
    # Bu kod bloğu sadece eğitim bittikten sonra çalışacaktır.
    try:
        # Eğer eğitim bittiyse ve adaptör kaydedildiyse:
        inference_model, tokenizer = setup_inference_model(ADAPTER_SAVE_PATH)
    except Exception as e:
        print(f"\nUYARI: Qwen adaptörü yüklenemedi. Önce eğitimi başlatmanız ve kaydetmeniz gerekir. Hata: {e}")
        # Hata durumunda sadece Retrieval modelini kullanarak testi yapabilirsiniz (metin üretimi olmadan).
        # Veya buraya eğitim tamamlanmış modelin manuel yükleme kodunu koyabilirsiniz.
        exit() # Örnek olması için durduruyoruz.

    # --- 4. RAG Sistemi Testi ---
    test_query = "Ticari satımdan kaynaklanan bir davada kötü niyet tazminatı talebi hangi gerekçeyle reddedilir?"

    final_answer = hukuki_rag_sistemi(
        query=test_query,
        inference_model=inference_model,
        tokenizer=tokenizer,
        retrieval_model=retrieval_model,
        index=faiss_index,
        corpus_dataset=corpus_dataset
    )

    print("\n\n=============== HUKUKİ ASİSTAN NIHAI YANITI ===============")
    print(final_answer)
    print("=========================================================")

Mounted at /content/drive
--- 1. Veri Seti Yükleme ---
**BULUNAN DOSYA SAYISI:** 1000
Veri Hazırlığı Tamamlandı. Dataset boyutu: 1000 örnek.

--- 2. Erişim Sistemi Kurulumu (XLM-R/FAISS) ---
Retrieval Modeli yüklendi: paraphrase-multilingual-mpnet-base-v2


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Vektör boyutu: (1000, 768)
FAISS İndeksi başarıyla oluşturuldu ve 'hukuk_faiss_index.bin' adresine kaydedildi.


config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.54G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# LLM ve QLoRA için gerekli kütüphaneler
!pip install transformers accelerate bitsandbytes peft trl

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer

# -----------------------------------------------------------
# 1. Veri Setini LLM Eğitim Formatına Dönüştürme
# -----------------------------------------------------------
def create_instruction_format(row):
    """Karar verisini Soru-Yanıt (Instruction-Output) formatına dönüştürür."""
    # Qwen'i eğitmek istediğimiz görev: Kararı analiz edip özetlemek
    instruction = f"Aşağıdaki hukuki kararı analiz ederek, davaya konu olayı, mahkeme gerekçesini ve hükmünü hukuki terminolojiye uygun bir şekilde özetle."

    # Çıktı (Output) metnini oluşturma
    output = (
        f"DAVANIN KONUSU: {row['meta_data']['case_subject']}\n\n"
        f"OLAY VE TALEPLER: {row['rrl_segments']['facts_text']}\n\n"
        f"MAHKEME GEREKÇESİ: {row['rrl_segments']['reasoning_text']}\n\n"
        f"HÜKÜM (VERDİCT): {row['rrl_segments']['verdict_text']}"
    )

    # LLM'in eğitimi için son format
    return {"text": f"### HUKUKİ YÖNERGE:\n{instruction}\n\n### YANIT:\n{output}"}

# Veri setini LLM için hazırlama
training_data = corpus_dataset.map(create_instruction_format)
print("LLM Eğitim Veri Seti Hazırlandı.")


# -----------------------------------------------------------
# 2. Qwen Modeli ve QLoRA Ayarları (Hukuki Fine-Tuning)
# -----------------------------------------------------------
model_id = "Qwen/Qwen1.5-7B-Chat" # 7 milyar parametreli Qwen modeli

# 4-bit Kuantizasyon Ayarları (QLoRA)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print(f"2. Qwen modeli ({model_id}) 4-bit kuantizasyon ile yükleniyor...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # GPU'nuz varsa otomatik kullanır
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # Qwen için tokenizer ayarı

# QLoRA (LoRA) ayarları
lora_config = LoraConfig(
    r=16, # Rank: Ne kadar adaptasyon yapılacağı. 16 iyi bir başlangıçtır.
    lora_alpha=32,
    target_modules="all-linear", # Qwen'deki tüm uygun katmanları eğit
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# -----------------------------------------------------------
# 3. Eğitimi Başlatma
# -----------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./qlora_hukuk_qwen_sonuc",
    num_train_epochs=3, # 3 Epoch, 1000 örnek için yeterli olabilir
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    save_strategy="epoch",
    logging_steps=10
)

trainer = SFTTrainer(
    model=model,
    train_dataset=training_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=2048, # Hukuki metinlerin uzunluğuna göre ayarlandı
    tokenizer=tokenizer,
    args=training_args,
)

print("\n--- QLoRA Hukuk Eğitimi Başlatılıyor ---")
# trainer.train() # Bu komut ile eğitim başlar.
# NOT: Bu işlem, GPU gerektirir ve Colab'de saatler sürebilir.

# Eğitimi bitirdikten sonra adaptör ağırlıklarını kaydedin:
# trainer.model.save_pretrained("./hukuk_qwen_adaptoru")
# tokenizer.save_pretrained("./hukuk_qwen_adaptoru")